# Tugas 2: Analisis Frekuensi dan Desain Filter
## Frequency Analysis and Filter Design

**Mata Kuliah:** Pengolahan Sinyal Medis -- Universitas Indonesia

**Tenggat:** Akhir Minggu 4

---

| | |
|---|---|
| **Nama Mahasiswa** | *(isi nama Anda di sini)* |
| **NPM** | *(isi NPM Anda di sini)* |

---

### Petunjuk Pengerjaan

1. Lengkapi semua bagian yang ditandai dengan `# TODO`.
2. Jawab semua pertanyaan analisis pada sel Markdown yang telah disediakan.
3. Pastikan semua sel kode dapat dijalankan dari awal sampai akhir tanpa error (`Kernel > Restart & Run All`).
4. Kumpulkan notebook ini (file `.ipynb`) sesuai instruksi yang diberikan.

### Kriteria Penilaian

| Komponen | Bobot |
|---|---|
| Bagian 1: Analisis Spektrum FFT | 20% |
| Bagian 2: PSD dan Spectrogram | 20% |
| Bagian 3: Desain Filter IIR | 20% |
| Bagian 4: Desain Filter FIR | 20% |
| Pertanyaan Analisis | 20% |

In [ ]:
# Impor library yang dibutuhkan
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig

from medsinyal.io import load_synthetic
from medsinyal.filters import (
    bandpass_filter,
    lowpass_filter,
    highpass_filter,
    notch_filter,
    remove_baseline_wander,
    design_fir_bandpass,
    apply_fir,
)

# Konfigurasi plot
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Seed untuk reproducibility
np.random.seed(42)

---
## Bagian 1: Analisis Spektrum FFT (20%)
### FFT Spectrum Analysis

**Fast Fourier Transform (FFT)** adalah algoritma efisien untuk menghitung **Discrete Fourier Transform (DFT)** yang mengubah sinyal dari domain waktu ke domain frekuensi:

$$X[k] = \sum_{n=0}^{N-1} x[n] \cdot e^{-j 2\pi kn / N}$$

Dari hasil FFT, kita dapat menghitung **spektrum magnitude** yang menunjukkan kekuatan (amplitudo) setiap komponen frekuensi dalam sinyal. Untuk sinyal nyata (real-valued), spektrum FFT bersifat simetri sehingga cukup digunakan **spektrum satu sisi** (*one-sided spectrum*) untuk frekuensi $0$ sampai $f_s/2$ (Nyquist frequency).

Magnitude yang dinormalisasi untuk spektrum satu sisi:
$$|X_\text{one-sided}[k]| = \frac{2}{N} |X[k]|, \quad k = 1, 2, \ldots, N/2 - 1$$

### Tugas 1.1: FFT Sinyal Multi-Frekuensi

Muat data `sine_compositions` dan hitung **spektrum FFT magnitude satu sisi** dari sinyal `multi_freq`. Identifikasi semua komponen frekuensi yang ada berdasarkan puncak (*peaks*) yang terlihat pada spektrum.

In [ ]:
# TODO: Muat data sine_compositions
# data = load_synthetic('sine_compositions')
# t = data['t']
# fs = float(data['fs'])
# sinyal = data['multi_freq']
# print(f"Sampling rate: {fs} Hz")
# print(f"Durasi sinyal: {t[-1]:.2f} detik")
# print(f"Jumlah sampel: {len(sinyal)}")

# TODO: Hitung FFT
# N = len(sinyal)
# fft_result = np.fft.fft(sinyal)
# freq = np.fft.fftfreq(N, d=1/fs)

# TODO: Ambil spektrum satu sisi (one-sided)
# Ambil setengah pertama (frekuensi positif saja)
# half = N // 2
# freq_pos = freq[:half]
# magnitude = (2/N) * np.abs(fft_result[:half])

# TODO: Plot spektrum magnitude
# plt.figure(figsize=(12, 4))
# plt.plot(freq_pos, magnitude)
# plt.xlabel('Frekuensi (Hz)')
# plt.ylabel('Magnitude')
# plt.title('Spektrum FFT: Komposisi Sinyal Multi-Frekuensi')
# plt.xlim([0, fs/2])
# plt.show()

# TODO: Identifikasi puncak frekuensi menggunakan scipy.signal.find_peaks
# peaks, _ = sig.find_peaks(magnitude, height=0.05)
# print("Komponen frekuensi yang teridentifikasi:")
# for p in peaks:
#     print(f"  f = {freq_pos[p]:.1f} Hz, Magnitude = {magnitude[p]:.4f}")


### Tugas 1.2: Analisis Spektrum FFT ECG Bernoise

Muat data `synthetic_ecg`, hitung spektrum FFT dari `ecg_noisy`, dan bandingkan dengan spektrum `ecg_clean`. Identifikasi:
- Di frekuensi berapa interferensi jala-jala listrik (*power line interference*) muncul?
- Apa kisaran frekuensi komponen QRS kompleks pada ECG?
- Bagaimana perbedaan spektrum ECG bersih versus bernoise?

In [ ]:
# Muat data ECG sintetik
ecg_data = load_synthetic('synthetic_ecg')
t_ecg = ecg_data['t']
fs_ecg = float(ecg_data['fs'])
ecg_clean = ecg_data['ecg_clean']
ecg_noisy = ecg_data['ecg_noisy']

print(f"Sampling rate ECG: {fs_ecg} Hz")
print(f"Durasi: {t_ecg[-1]:.2f} detik")
print(f"Jumlah sampel: {len(ecg_clean)}")

In [ ]:
# TODO: Hitung FFT untuk ecg_clean dan ecg_noisy
# N_ecg = len(ecg_clean)
# freq_ecg = np.fft.fftfreq(N_ecg, d=1/fs_ecg)
# half_ecg = N_ecg // 2
# freq_ecg_pos = freq_ecg[:half_ecg]

# fft_clean = np.fft.fft(ecg_clean)
# mag_clean = (2/N_ecg) * np.abs(fft_clean[:half_ecg])

# fft_noisy = np.fft.fft(ecg_noisy)
# mag_noisy = (2/N_ecg) * np.abs(fft_noisy[:half_ecg])

# TODO: Plot perbandingan spektrum ECG bersih vs bernoise
# fig, axes = plt.subplots(2, 1, figsize=(12, 7), sharex=True)

# axes[0].plot(freq_ecg_pos, mag_clean, 'b-')
# axes[0].set_title('Spektrum FFT: ECG Bersih')
# axes[0].set_ylabel('Magnitude')
# axes[0].set_xlim([0, 150])

# axes[1].plot(freq_ecg_pos, mag_noisy, 'r-')
# axes[1].set_title('Spektrum FFT: ECG Bernoise')
# axes[1].set_ylabel('Magnitude')
# axes[1].set_xlabel('Frekuensi (Hz)')
# axes[1].set_xlim([0, 150])

# # Tandai frekuensi 50 Hz pada subplot bernoise
# axes[1].axvline(x=50, color='orange', linestyle='--', label='50 Hz (interferensi jala-jala)')
# axes[1].legend()

# plt.tight_layout()
# plt.show()


---
## Bagian 2: PSD dan Spectrogram (20%)
### Power Spectral Density and Spectrogram

**Power Spectral Density (PSD)** menggambarkan distribusi daya (*power*) sinyal terhadap frekuensi. Metode **Welch** mengestimasi PSD dengan membagi sinyal menjadi segmen-segmen yang overlapping, menghitung periodogram tiap segmen, kemudian merata-ratakannya untuk mengurangi varians:

$$P_{\text{Welch}}(f) = \frac{1}{K} \sum_{k=0}^{K-1} |X_k(f)|^2$$

**Spectrogram** adalah representasi PSD yang berubah terhadap waktu (*time-frequency representation*) menggunakan **Short-Time Fourier Transform (STFT)**. Spectrogram berguna untuk menganalisis sinyal non-stasioner seperti EEG yang konten frekuensinya berubah terhadap waktu.

Pita frekuensi EEG yang relevan secara klinis:

| Pita | Rentang Frekuensi | Asosiasi |
|---|---|---|
| Delta ($\delta$) | 0.5 -- 4 Hz | Tidur dalam, koma |
| Theta ($\theta$) | 4 -- 8 Hz | Mengantuk, meditasi |
| Alpha ($\alpha$) | 8 -- 13 Hz | Relaksasi, mata tertutup |
| Beta ($\beta$) | 13 -- 30 Hz | Terjaga, konsentrasi |
| Gamma ($\gamma$) | 30 -- 50 Hz | Kognitif tinggi |

### Tugas 2.1: PSD EEG dengan Metode Welch

Muat data `synthetic_eeg` dan hitung **PSD sinyal EEG bersih** menggunakan metode Welch. Tandai batas-batas pita frekuensi EEG pada plot PSD.

In [ ]:
# Muat data EEG sintetik
eeg_data = load_synthetic('synthetic_eeg')

print("Keys dalam file:", list(eeg_data.keys()))
for key in eeg_data:
    if isinstance(eeg_data[key], np.ndarray):
        print(f"  {key}: shape={eeg_data[key].shape}, dtype={eeg_data[key].dtype}")
    else:
        print(f"  {key}: {eeg_data[key]}")

In [ ]:
# Ambil sinyal EEG dari data
t_eeg = eeg_data['t']
fs_eeg = float(eeg_data['fs'])
eeg_clean = eeg_data['eeg_clean']
eeg_noisy = eeg_data['eeg_noisy']

# TODO: Hitung PSD dengan metode Welch
# f_psd, psd = sig.welch(eeg_clean, fs_eeg, nperseg=256)

# TODO: Plot PSD dengan skala semilog (plt.semilogy)
# plt.figure(figsize=(12, 5))
# plt.semilogy(f_psd, psd, 'b-')
# plt.xlabel('Frekuensi (Hz)')
# plt.ylabel('PSD (V²/Hz)')
# plt.title('Power Spectral Density EEG (Metode Welch)')
# plt.xlim([0, 60])

# TODO: Tandai batas pita frekuensi dengan garis vertikal
# band_limits = [0.5, 4, 8, 13, 30, 50]
# band_names = ['', 'Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']
# for freq_limit in [4, 8, 13, 30, 50]:
#     plt.axvline(x=freq_limit, color='red', linestyle='--', alpha=0.5)

# TODO: Tambahkan label nama pita di tengah setiap rentang
# Gunakan plt.text() untuk menambahkan teks 'Delta', 'Theta', dst.
# Contoh: plt.text(2, ..., 'Delta', ha='center')

# plt.show()


### Tugas 2.2: Spectrogram EEG

Hitung dan visualisasikan **spectrogram** dari sinyal EEG. Perhatikan bagaimana pita alpha (8--13 Hz) mungkin tampak lebih menonjol pada rentang waktu tertentu dalam data sintetik (region alpha burst).

In [ ]:
# TODO: Hitung spectrogram menggunakan scipy.signal.spectrogram
# f_spec, t_spec, Sxx = sig.spectrogram(eeg_clean, fs_eeg, nperseg=256, noverlap=128)

# TODO: Plot spectrogram
# plt.figure(figsize=(14, 5))
# plt.pcolormesh(t_spec, f_spec, 10 * np.log10(Sxx + 1e-12), shading='gouraud', cmap='inferno')
# plt.colorbar(label='Daya (dB)')
# plt.ylabel('Frekuensi (Hz)')
# plt.xlabel('Waktu (detik)')
# plt.title('Spectrogram EEG')
# plt.ylim([0, 60])

# TODO: Tandai batas pita frekuensi dengan garis horizontal
# for freq_limit in [4, 8, 13, 30]:
#     plt.axhline(y=freq_limit, color='white', linestyle='--', alpha=0.6, linewidth=0.8)

# plt.show()

# TODO: Plot perbandingan sinyal EEG domain waktu di atas spectrogram
# fig, axes = plt.subplots(2, 1, figsize=(14, 8))
# axes[0].plot(t_eeg, eeg_clean, 'b-', linewidth=0.5)
# axes[0].set_title('Sinyal EEG Bersih (Domain Waktu)')
# axes[0].set_ylabel('Amplitudo (µV)')
# axes[0].set_xlabel('Waktu (detik)')
# # (ulangi plot spectrogram pada axes[1])
# plt.tight_layout()
# plt.show()


---
## Bagian 3: Desain Filter IIR (20%)
### IIR Filter Design

Filter **IIR (Infinite Impulse Response)** memiliki umpan balik (*feedback*) sehingga respon impulsnya berlangsung tak terbatas. Filter ini umumnya lebih efisien secara komputasi dibanding FIR untuk spesifikasi yang sama, namun memiliki **fase non-linear**.

**Filter Butterworth** adalah salah satu jenis IIR yang paling umum digunakan. Karakteristiknya:
- Respon magnitudo **maximally flat** di *passband* (tidak ada ripple)
- Roll-off semakin tajam dengan meningkatnya orde filter
- Frekuensi cutoff $f_c$ didefinisikan pada titik **-3 dB** (daya turun setengah)

Respon magnitudo filter Butterworth orde-$n$:
$$|H(j\omega)|^2 = \frac{1}{1 + \left(\omega / \omega_c\right)^{2n}}$$

### Tugas 3.1: Desain dan Visualisasi Respon Frekuensi Filter Butterworth

Desain filter **Butterworth bandpass** untuk ECG dengan spesifikasi:
- Passband: 0.5 -- 40 Hz
- Sampling rate: 500 Hz
- Orde filter: 4

Plot respon frekuensinya (magnitude dalam dB) dan tandai titik -3 dB.

In [ ]:
fs = fs_ecg  # Gunakan sampling rate ECG (500 Hz)

# TODO: Desain filter Butterworth bandpass
# nyq = fs / 2          # Nyquist frequency
# low = 0.5 / nyq      # Normalisasi frekuensi cutoff bawah
# high = 40.0 / nyq    # Normalisasi frekuensi cutoff atas
# b_iir, a_iir = sig.butter(4, [low, high], btype='band')

# TODO: Hitung respon frekuensi menggunakan freqz
# w_iir, h_iir = sig.freqz(b_iir, a_iir, worN=2000, fs=fs)

# TODO: Plot respon frekuensi dalam dB
# plt.figure(figsize=(12, 5))
# plt.plot(w_iir, 20 * np.log10(np.abs(h_iir) + 1e-12), 'b-', label='Butterworth IIR (orde 4)')
# plt.axhline(y=-3, color='red', linestyle='--', label='-3 dB cutoff')
# plt.axvline(x=0.5, color='green', linestyle=':', alpha=0.7, label='f_low = 0.5 Hz')
# plt.axvline(x=40.0, color='green', linestyle=':', alpha=0.7, label='f_high = 40 Hz')
# plt.xlabel('Frekuensi (Hz)')
# plt.ylabel('Magnitude (dB)')
# plt.title('Respon Frekuensi: Butterworth Bandpass (0.5-40 Hz)')
# plt.xlim([0, fs/2])
# plt.ylim([-80, 5])
# plt.legend()
# plt.show()


### Tugas 3.2: Terapkan Filter IIR pada ECG Bernoise

Terapkan filter bandpass dan notch filter pada `ecg_noisy`. Bandingkan:
1. Sinyal asli bernoise
2. Setelah bandpass filter (0.5--40 Hz)
3. Setelah notch filter 50 Hz + bandpass filter

In [ ]:
# TODO: Terapkan bandpass filter pada ECG noisy menggunakan medsinyal.filters
# ecg_bp = bandpass_filter(ecg_noisy, 0.5, 40.0, fs_ecg)

# TODO: Terapkan notch filter 50 Hz terlebih dahulu, kemudian bandpass
# ecg_notched = notch_filter(ecg_noisy, 50.0, fs_ecg)
# ecg_notch_bp = bandpass_filter(ecg_notched, 0.5, 40.0, fs_ecg)

# TODO: Plot perbandingan ketiga sinyal dalam 3 subplot
# Gunakan 2-3 detik pertama agar detail gelombang terlihat jelas
# t_show = 3.0  # detik
# mask = t_ecg <= t_show

# fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# axes[0].plot(t_ecg[mask], ecg_noisy[mask], 'r-', linewidth=0.7)
# axes[0].set_title('ECG Asli Bernoise')
# axes[0].set_ylabel('Amplitudo (mV)')

# axes[1].plot(t_ecg[mask], ecg_clean[mask], 'b-', alpha=0.5, label='ECG Bersih')
# axes[1].plot(t_ecg[mask], ecg_bp[mask], 'g-', label='Setelah Bandpass (0.5-40 Hz)')
# axes[1].set_title('ECG Setelah Bandpass Filter IIR')
# axes[1].set_ylabel('Amplitudo (mV)')
# axes[1].legend(loc='upper right')

# axes[2].plot(t_ecg[mask], ecg_clean[mask], 'b-', alpha=0.5, label='ECG Bersih')
# axes[2].plot(t_ecg[mask], ecg_notch_bp[mask], 'm-', label='Setelah Notch + Bandpass')
# axes[2].set_title('ECG Setelah Notch Filter 50 Hz + Bandpass')
# axes[2].set_ylabel('Amplitudo (mV)')
# axes[2].set_xlabel('Waktu (detik)')
# axes[2].legend(loc='upper right')

# plt.tight_layout()
# plt.show()


### Tugas 3.3: Ekstraksi Pita Alpha EEG dengan Filter IIR

Terapkan **bandpass filter** untuk mengekstraksi pita alpha (8--13 Hz) dari sinyal `eeg_noisy`. Bandingkan dengan sinyal `band_alpha` (ground truth) dari data sintetik.

In [ ]:
# Ambil ground truth pita alpha
band_alpha_gt = eeg_data['band_alpha']

# TODO: Terapkan bandpass filter untuk mengekstraksi pita alpha (8-13 Hz)
# alpha_extracted = bandpass_filter(eeg_noisy, 8.0, 13.0, fs_eeg)

# TODO: Hitung RMSE antara extracted alpha dan ground truth
# rmse_alpha = np.sqrt(np.mean((alpha_extracted - band_alpha_gt) ** 2))
# print(f"RMSE Alpha Extracted vs Ground Truth: {rmse_alpha:.6f}")

# TODO: Plot perbandingan sinyal alpha ground truth dan hasil filter
# t_show_eeg = 5.0  # tampilkan 5 detik pertama
# mask_eeg = t_eeg <= t_show_eeg

# fig, axes = plt.subplots(2, 1, figsize=(14, 7), sharex=True)

# axes[0].plot(t_eeg[mask_eeg], band_alpha_gt[mask_eeg], 'b-', label='Alpha Ground Truth')
# axes[0].set_title('Pita Alpha EEG: Ground Truth')
# axes[0].set_ylabel('Amplitudo (µV)')
# axes[0].legend()

# axes[1].plot(t_eeg[mask_eeg], alpha_extracted[mask_eeg], 'r-', label='Alpha Hasil Filter IIR')
# axes[1].set_title(f'Pita Alpha EEG: Hasil Bandpass Filter (8-13 Hz), RMSE={rmse_alpha:.4f}')
# axes[1].set_ylabel('Amplitudo (µV)')
# axes[1].set_xlabel('Waktu (detik)')
# axes[1].legend()

# plt.tight_layout()
# plt.show()


---
## Bagian 4: Desain Filter FIR (20%)
### FIR Filter Design

Filter **FIR (Finite Impulse Response)** tidak memiliki umpan balik (*feedback*) sehingga respon impulsnya memiliki panjang yang terbatas. Keunggulan utama filter FIR:
- **Fase linear** -- tidak ada distorsi fase, penting untuk sinyal biomedis
- **Selalu stabil** -- karena tidak ada pole di luar lingkaran satuan
- Mudah didesain dengan metode **windowing**

Metode windowing mendesain FIR dengan mengalikan respon impuls ideal (sinc function) dengan sebuah fungsi window (misalnya **Hamming window**):

$$h[n] = h_d[n] \cdot w[n], \quad n = 0, 1, \ldots, M$$

di mana $M+1$ adalah panjang filter (*numtaps*), dan $h_d[n]$ adalah respon impuls ideal.

**Kompromi utama FIR vs IIR:**

| Aspek | FIR | IIR |
|---|---|---|
| Fase | Linear (tanpa distorsi) | Non-linear |
| Stabilitas | Selalu stabil | Bisa tidak stabil |
| Efisiensi | Butuh banyak koefisien | Lebih efisien |
| Ripple | Bisa dikontrol | Bergantung tipe |
| Cocok untuk | Sinyal biomedis, audio | Sinyal real-time |

### Tugas 4.1: Desain Filter FIR dan Perbandingan Respon Frekuensi

Desain filter **FIR bandpass** untuk ECG menggunakan `design_fir_bandpass` dari `medsinyal.filters`. Bandingkan respon frekuensinya dengan filter Butterworth IIR dari Tugas 3.1 dalam satu plot.

In [ ]:
# TODO: Desain filter FIR bandpass untuk ECG (0.5-40 Hz, 101 taps)
# fir_coefs = design_fir_bandpass(0.5, 40.0, fs_ecg, numtaps=101)
# print(f"Jumlah koefisien FIR (numtaps): {len(fir_coefs)}")

# TODO: Hitung respon frekuensi FIR menggunakan freqz
# w_fir, h_fir = sig.freqz(fir_coefs, [1.0], worN=2000, fs=fs_ecg)

# TODO: Desain ulang filter IIR (atau gunakan b_iir, a_iir dari Tugas 3.1)
# nyq = fs_ecg / 2
# b_iir, a_iir = sig.butter(4, [0.5/nyq, 40.0/nyq], btype='band')
# w_iir, h_iir = sig.freqz(b_iir, a_iir, worN=2000, fs=fs_ecg)

# TODO: Plot perbandingan respon frekuensi FIR vs IIR dalam satu figure
# fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# # Subplot 1: Magnitude response
# axes[0].plot(w_iir, 20 * np.log10(np.abs(h_iir) + 1e-12), 'b-', label='IIR Butterworth (orde 4)')
# axes[0].plot(w_fir, 20 * np.log10(np.abs(h_fir) + 1e-12), 'r-', label='FIR Hamming (101 taps)')
# axes[0].axhline(y=-3, color='gray', linestyle='--', alpha=0.7, label='-3 dB')
# axes[0].set_xlabel('Frekuensi (Hz)')
# axes[0].set_ylabel('Magnitude (dB)')
# axes[0].set_title('Perbandingan Respon Frekuensi: FIR vs IIR Bandpass (0.5-40 Hz)')
# axes[0].set_xlim([0, fs_ecg/2])
# axes[0].set_ylim([-80, 5])
# axes[0].legend()

# # Subplot 2: Phase response
# axes[1].plot(w_iir, np.unwrap(np.angle(h_iir)), 'b-', label='IIR Butterworth')
# axes[1].plot(w_fir, np.unwrap(np.angle(h_fir)), 'r-', label='FIR Hamming')
# axes[1].set_xlabel('Frekuensi (Hz)')
# axes[1].set_ylabel('Fase (radian)')
# axes[1].set_title('Perbandingan Respon Fase: FIR vs IIR')
# axes[1].set_xlim([0, 60])
# axes[1].legend()

# plt.tight_layout()
# plt.show()


### Tugas 4.2: Terapkan Filter FIR dan Bandingkan dengan IIR

Terapkan filter FIR pada `ecg_noisy` menggunakan `apply_fir`. Bandingkan hasil filter FIR dengan filter IIR dari Tugas 3.2. Hitung **RMSE** keduanya terhadap `ecg_clean` -- filter mana yang memberikan hasil lebih baik?

In [ ]:
# TODO: Terapkan filter FIR pada ECG noisy
# ecg_fir = apply_fir(ecg_noisy, fir_coefs)

# TODO: Gunakan hasil filter IIR dari Tugas 3.2 (ecg_bp atau ecg_notch_bp)
# Jika belum ada, hitung ulang:
# ecg_iir = bandpass_filter(ecg_noisy, 0.5, 40.0, fs_ecg)

# TODO: Hitung RMSE FIR dan IIR terhadap ecg_clean
# rmse_fir = np.sqrt(np.mean((ecg_fir - ecg_clean) ** 2))
# rmse_iir = np.sqrt(np.mean((ecg_iir - ecg_clean) ** 2))
# rmse_noisy = np.sqrt(np.mean((ecg_noisy - ecg_clean) ** 2))

# print(f"RMSE ECG Noisy (tanpa filter) : {rmse_noisy:.6f}")
# print(f"RMSE IIR Butterworth           : {rmse_iir:.6f}")
# print(f"RMSE FIR Hamming               : {rmse_fir:.6f}")
# print()
# if rmse_fir < rmse_iir:
#     print("=> Filter FIR menghasilkan RMSE lebih rendah (lebih baik)")
# else:
#     print("=> Filter IIR menghasilkan RMSE lebih rendah (lebih baik)")

# TODO: Plot perbandingan hasil filtering dalam 3 subplot (tampilkan 3 detik pertama)
# t_show = 3.0
# mask = t_ecg <= t_show

# fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# axes[0].plot(t_ecg[mask], ecg_clean[mask], 'b-', alpha=0.6, label='ECG Bersih (referensi)')
# axes[0].plot(t_ecg[mask], ecg_noisy[mask], 'r-', alpha=0.6, label='ECG Noisy')
# axes[0].set_title('ECG Noisy vs ECG Bersih')
# axes[0].set_ylabel('Amplitudo (mV)')
# axes[0].legend(loc='upper right')

# axes[1].plot(t_ecg[mask], ecg_clean[mask], 'b-', alpha=0.5, label='ECG Bersih')
# axes[1].plot(t_ecg[mask], ecg_iir[mask], 'g-', label=f'Filter IIR (RMSE={rmse_iir:.4f})')
# axes[1].set_title('Hasil Filter Butterworth IIR')
# axes[1].set_ylabel('Amplitudo (mV)')
# axes[1].legend(loc='upper right')

# axes[2].plot(t_ecg[mask], ecg_clean[mask], 'b-', alpha=0.5, label='ECG Bersih')
# axes[2].plot(t_ecg[mask], ecg_fir[mask], 'm-', label=f'Filter FIR (RMSE={rmse_fir:.4f})')
# axes[2].set_title('Hasil Filter FIR Hamming')
# axes[2].set_ylabel('Amplitudo (mV)')
# axes[2].set_xlabel('Waktu (detik)')
# axes[2].legend(loc='upper right')

# plt.tight_layout()
# plt.show()


---
## Pertanyaan Analisis (20%)
### Analysis Questions

Jawab pertanyaan-pertanyaan berikut berdasarkan pemahaman Anda dari tugas di atas dan materi kuliah. Jawab dengan lengkap dan jelas.

### Pertanyaan 1

Dari spektrum FFT ECG bernoise (Tugas 1.2), jelaskan:

**(a)** Pada frekuensi berapa interferensi jala-jala listrik (*power line interference*) muncul, dan mengapa frekuensi tersebut berbeda di beberapa negara (misalnya Indonesia vs Amerika Serikat)?

**(b)** Mengapa komponen interferensi 50 Hz ini berbahaya untuk diagnosis ECG, padahal frekuensi QRS kompleks jauh lebih rendah?

**(c)** Bagaimana **notch filter** mengatasi masalah ini tanpa merusak komponen sinyal ECG yang penting? Jelaskan prinsip kerjanya.

**Jawaban:**

*(Tulis jawaban Anda di sini)*


### Pertanyaan 2

Jelaskan perbedaan mendasar antara filter **IIR** dan **FIR** dalam hal:

**(a)** Respon impuls (*impulse response*) -- apa yang dimaksud dengan "finite" dan "infinite"?

**(b)** Respon fase (*phase response*) -- mengapa FIR memiliki fase linear sedangkan IIR tidak?

**(c)** Stabilitas (*stability*) -- mengapa FIR selalu stabil sedangkan IIR bisa tidak stabil?

**(d)** Berdasarkan pengamatan Anda pada Tugas 4.2, kapan lebih baik menggunakan filter FIR daripada IIR untuk sinyal biomedis? Sebutkan contoh aplikasi klinis yang konkret.

**Jawaban:**

*(Tulis jawaban Anda di sini)*


### Pertanyaan 3

Berdasarkan analisis PSD EEG menggunakan metode Welch (Tugas 2.1):

**(a)** Pita frekuensi mana yang memiliki daya (*power*) terbesar pada sinyal EEG sintetik? Mengapa demikian secara fisiologis?

**(b)** Bagaimana PSD EEG dapat digunakan untuk mendeteksi perubahan kondisi mental seseorang, misalnya dari **terjaga** ke **mengantuk** ke **tidur nyenyak**? Pita frekuensi apa yang meningkat/menurun pada setiap transisi?

**(c)** Apa keunggulan **spectrogram** dibandingkan PSD biasa untuk menganalisis perubahan kondisi mental yang terjadi secara bertahap?

**Jawaban:**

*(Tulis jawaban Anda di sini)*


---

## Checklist Sebelum Pengumpulan

Pastikan Anda telah:

- [ ] Mengisi nama dan NPM di bagian header
- [ ] Menyelesaikan semua bagian kode yang ditandai `# TODO`
- [ ] Menjawab semua pertanyaan analisis
- [ ] Menjalankan seluruh notebook dari awal (`Kernel > Restart & Run All`) tanpa error
- [ ] Semua plot tampil dengan benar dan memiliki label sumbu serta judul

---
*Tugas 2 -- Pengolahan Sinyal Medis -- Universitas Indonesia*